In [1]:
import random
import copy
import numpy as np
import pyspiel
from open_spiel.python.algorithms import mcts, random_agent
from othello_game import OthelloGame, OthelloState
from MCTS import MCTS
from BasicModel import BasicModel
from NNet import NNetWrapper
from QNet import QNetWrapper
from HybridQNet import HybridQNetWrapper
from Arena import Arena
from Coach import Coach

In [2]:
game = pyspiel.load_game("othello_nxn")
# nnet = NNetWrapper(game)
nnet = QNetWrapper(game)
# nnet.load_checkpoint()

In [3]:
# nnet.load_checkpoint("./temp", "checkpoint_61.pth.tar")
coach = Coach(game, nnet)
coach.learn()

Iter 1


Self Play: 100%|██████████| 5/5 [00:05<00:00,  1.13s/it]


Epoch 0


Training Net: 100%|██████████| 4/4 [00:06<00:00,  1.68s/it, Loss_v=1.02e+00]


Epoch 1


...

Pitting against previous version
Iter 4


Self Play: 100%|██████████| 5/5 [00:05<00:00,  1.17s/it]


Epoch 0


Training Net:  31%|███▏      | 5/16 [00:08<00:18,  1.70s/it, Loss_v=7.71e-01]


KeyboardInterrupt: 

In [4]:
def get_alpha_zero_player():
    class AlphaZeroPlayer:
        def __init__(self, game):
            self.game = game
            self.mcts = MCTS(game, nnet, numMCTSSims=10)

        def step(self, state):
            probs = self.mcts.getActionProb(state, temp=1)
            action = np.argmax(probs)
            return action

    return AlphaZeroPlayer(game)

def get_random_player():
    class RandomPlayer:
        def __init__(self, game):
            self.game = game

        def step(self, state):
            legal_actions = state.legal_actions()
            action = random.choice(legal_actions)
            return action

    return RandomPlayer(game)

def get_mcts_player():
    class MCTSPlayer:
        def __init__(self, game):
            self.game = game
            evaluator = mcts.RandomRolloutEvaluator(n_rollouts=3)
            self.mcts = mcts.MCTSBot(game, uct_c=2, max_simulations=25, evaluator=evaluator)

        def step(self, state):
            action = self.mcts.step(state)
            return action

    return MCTSPlayer(game)

arena = Arena(game, get_alpha_zero_player, get_random_player)
# arena = Arena(game, get_alpha_zero_player, get_mcts_player)
arena.playGames(50, verbose=True)

# (31, 65, 4)
# (31, 62, 7)

Arena.playGames (1):  70%|███████   | 7/10 [00:05<00:02,  1.27it/s]